# **Metabolomics Data Visualisation Workflow for PLS-DA (PD-CG) [W/ SMOTE]**

This Google Colab notebook describes a metabolomics data analysis and visualisation workflow for partial least squares regression (a.k.a. projection to latent structure) with a binary classification outcome.

This computational workflow is described using a previously published LC-MS dataset by Sinclair et al. (2021). The study compared the metabolomic profiles across Parkinson's disease patients, characterised as medicated (MD; n=138) and drug-naive (DN; n=80), versus control (CG; n=56) using 8765 named metabolites. For the purpose of this computational workflow, only the (DN & MD) vs CG samples were compared in a binary discriminant analysis. The deconvolved and annotated data from this study is deposited on Metabolomics Workbench (Study ID: MTBLS2266)

This computational workflow requires a dataset to be in, or converted to, a previously described standardised Excel file format (Mendez et al. 2019). This format uses the Tidy Data Framework (Wickham, 2014), where each row represents an observation (e.g. sample) and each column represents a variable (e.g. age or metabolite). Each excel file (per study) contains two sheets; a data sheet and a peak sheet. The data sheet contains the metabolite concentration together with the metadata associated for each observation (requiring the inclusion of the columns: Idx, SampleID, and Class). The peak sheet contains the additional metadata that pertains to the metabolites in the data sheet (requiring the inclusion of the columns: Idx, Name, and Label). The standardisation of this format allows for the efficient re-use of this computational workflow.

The steps included in this data analysis and visualisation workflow are:
1. Import Packages 
2. Load Data and Peak Sheet
3. Data Pre-processing
4. Split Data into Train and Test Set
5. Hyperparameter Optimisation
6. Build Model and Evaluate
7. Permutation Testing
8. Bootstrap Resampling of the Model
9. 

# Section 1 - Import Packages

Certain packages need installing onto the virtual environment prior to use.

For this computational workflow the CIMCB package was installed using pip, CIMCB requires:

*   Python (>=3.5)
*   Bokeh (>=1.0.0)
*   Keras
*   NumPy (>=1.12)
*   SciPy
*   scikit-learn
*   Statsmodels
*   TensorFlow
*   tqdm

In [ ]:
pip install cimcb

To use tools that extend beyond the basic functionalities of Python programming, packages must first be imported to enable their use in each Google Colab environment. Each package is a container of modules.

For this computational workflow, the following packages were used:


*   numpy: A fundamental package for scientific computing with Python, primarly used for the manipulation of arrays
*   pandas: A fundamental package for data analysis and manipulation
*   cimcb: A package for the statistical analysis of untargeted and targeted metabolomics data
*   matplotlib.pyplot: A package mainly used for interactive plots and simple cases of programmatic plot generation
*   seaborn: A package that provides a high-level interface for drawing attractive and informative statistical graphics
*   sklearn: A fundamental package containing tools for machine learning
  *   train_test_split: A method to split arrays into training and test subsets

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import cimcb as cb

print('All packages successfully loaded')

All packages successfully loaded


In order to  reproducibility of the workflow, random seeds are set.

*   seed_split: Seed the generator using an integer value e.g. 42 (default = None ; no seed set)

This seed is used to mainatin duplicability in the way the data is divided when the train and test set are generated.

*  seed_init: seed the generator using an integer value e.g. 42 (default = None ; no seed set)

This seed is used to maintain duplicability in the way the intial weights are drawn from a truncated normal distribution when the neural network is first compiled.

In [ ]:
seed_split = 40
# seed_split = None

# Section 2 - Load Data & Peak Sheet

To upload the dataset to the Google Colab notebook environment, an upload widget was used.

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving Data and Peak sheet_Test.xlsx to Data and Peak sheet_Test.xlsx


The helper function load_dataXL loads the two data sheets from the Excel file 'Data and Peak sheet_Test.xlsx'. Provided the dataset adheres to the standardised TidyData framework format, load_dataXL() outputs the data sheets from the uploaded Excel file as individual Pandas DataFrames.

In [ ]:
# The path to the input file (Excel spreadsheet)
filename = 'Data and Peak sheet_Test.xlsx'
#filename = 'MTBLS290db.xlsx'

# Load Peak and Data tables into two variables
dataTable, peakTable = cb.utils.load_dataXL(filename, DataSheet='Data', PeakSheet='Peaks')

Loadings PeakFile: Peaks
Loadings DataFile: Data
Data Table & Peak Table is suitable.
TOTAL SAMPLES: 418 TOTAL PEAKS: 8765
Done!


# Section 3 - Data Pre-processing

**Section 3.1 - Data Cleaning**

According to Broadhurst (2019), it is best practice to access the quality of the data and refine the dataset by removing those metabolites that lack reporducible measurements. The QC-RSD and percentage of missing values has been calculated and are included in the peakTable DataFrame. Using those values, we remove all metabolomic features that do not meet the following criteria:

*   QC-RSD less than 20%
* Fewer than 10% of values are missing

In [ ]:
# Clean PeakTable and Extract PeakList
RSD = peakTable['QC_RSD']   
percMiss = peakTable['Perc_missing']  
peakTableClean = peakTable[(RSD < 20) & (percMiss < 10)]   
peakList = peakTableClean['Name']  

print("Number of peaks remaining: {}".format(len(peakTableClean)))

Number of peaks remaining: 2259


**Section 3.2 - Extract X and Y**

As previously mentioned, this workflow is performing binary classification of the sample types "Parkinson's Disease" and "Control Group". However, we must first balance the Classes "MD" and "DN" within the "Parkinson's Disease" sample type. The X matrix of metabolite concentrations and Y vector of classification labels ("MD"=0 and "DN"=1) are extracted through the following steps:

1.   Create a subset of the dataTable called dataTable1, containing samples in the Class "MD" or "DN"
2.   Use the peakList variable to hold the names of the metabolites to be used 
3.   Extract all the applicable columns, using peakList, from dataTable1 and place in matrix X
4.   Set Y to the list of binary outcomes from the "Class" column from dataTable1

In [ ]:
dataTable1 = dataTable[(dataTable.Class == "MD") | (dataTable.Class == "DN")]  # Reduce data table only to MD and DN class members
pos_outcome = "DN"

dataTable1['Class'] = [0 if x == 'MD' else 1 for x in dataTable1['Class']]

to_drop = ['Idx', 'SampleID']
dataTable1.drop(to_drop, axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
peaklist = peakTableClean['Name']          
X1 = dataTable1[peaklist]

merged = pd.concat([dataTable1['Class'], X1], axis=1)

merged.reset_index()

,index,Class,M86,M132,M137,M152,M155,M162,M164,M169,...,M8608,M8612,M8642,M8655,M8663,M8685,M8699,M8715,M8729,M8731
0,146,0,31119.367640,641540.06610,26298.038630,5.625178e+06,2.782152e+07,259.651238,483626.43890,7.368964e+04,...,51430.48636,18412.69619,122545.83210,4.453929e+05,145439.29730,6388.817039,93392.022820,296245.9963,265213.4407,39629.25380
1,147,0,27945.701480,106780.30810,19688.706950,8.018405e+05,9.672048e+06,1831.516812,198939.04160,3.946194e+04,...,35678.47362,10861.17415,239517.24310,8.471789e+05,236214.90110,5990.339636,1753.076374,324820.6171,358897.4505,56602.14902
2,148,0,1611.144812,110065.23280,18313.051570,1.243397e+05,2.038063e+06,435001.709000,44944.72675,2.021782e+04,...,65091.90226,19886.06258,893440.75840,1.808965e+06,295690.87760,87458.523660,6509.821039,725412.7526,498830.0383,591755.93280
3,149,0,50276.888560,75422.40729,27554.404720,5.704878e+05,4.083986e+06,358.799341,101763.97820,1.182281e+04,...,36318.98201,14386.00915,167610.44120,2.782122e+06,223798.55400,8321.018651,1731.627238,228220.1006,312055.6068,80658.81383
4,150,0,24708.489720,56223.63935,6637.978372,2.090515e+04,3.006743e+05,146992.465900,61455.45901,8.929524e+03,...,58034.91423,12801.76426,193332.88220,1.399533e+06,177408.84740,8224.423487,3191.840128,432866.7621,551678.4753,83748.00984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,414,1,36315.412600,86565.57846,23222.337100,3.317119e+04,5.598351e+05,395.919074,66311.67700,3.275502e+04,...,46122.78831,12739.39509,133219.76360,1.078761e+06,295381.02210,5199.881800,2443.107591,231510.2665,474812.9334,88244.10204
213,415,1,35866.811280,72788.15663,19592.272360,2.460379e+04,8.047924e+05,4205.444231,92143.49205,1.329979e+04,...,46576.65965,10894.21841,96108.91507,5.902008e+05,314446.36130,4286.765742,3827.322902,160844.7708,261013.7788,42853.80034
214,416,1,21391.177550,162137.88250,29672.810360,6.222119e+04,7.550895e+05,1751.395399,50674.20452,7.157885e+04,...,68105.74186,17894.81148,111495.47910,1.733569e+06,197707.73950,9569.443883,4396.332838,237807.3456,679614.8098,67960.37908
215,417,1,495.253371,41180.48082,15250.912450,8.606017e+05,7.962703e+06,26056.057370,180570.26800,2.219288e+04,...,59691.00440,17464.24978,302805.53720,1.230356e+06,255188.68980,9543.984383,2725.212169,397531.7394,927696.7840,75193.23502


**Section 3.3 - Transform and Scale Data**

The MinMaxScaler() method is used to scale all columns that contain values larger than 1, to [0,1] range. This makes the values more manageable when creating and evaluating of the model.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Scale only columns that have values greater than 1
to_scale = [col for col in X1.columns if X1[col].max() > 1]
mms = MinMaxScaler()
scaled = mms.fit_transform(merged[to_scale])
scaled = pd.DataFrame(scaled, columns=to_scale)

# Replace original columns with scaled ones
for col in scaled:
    merged.reset_index()[col] = scaled[col]

In [ ]:
y1 = merged['Class']
merged1 = pd.concat([y1.reset_index()['Class'], scaled], axis=1)

display(merged1)

from sklearn.model_selection import train_test_split

X = merged1.drop('Class', axis=1)
y = merged1['Class']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)

print(f'''% Positive class in Train = {np.round(y_train.value_counts(normalize=True)[1] * 100, 2)}
% Positive class in Test  = {np.round(y_test.value_counts(normalize=True)[1] * 100, 2)}''')

,Class,M86,M132,M137,M152,M155,M162,M164,M169,M173,...,M8608,M8612,M8642,M8655,M8663,M8685,M8699,M8715,M8729,M8731
0,0,0.071117,0.001317,0.449938,0.176988,0.145284,0.000426,0.133999,0.001260,0.000772,...,0.057720,0.303060,0.024880,0.000000,0.016097,0.052110,1.000000,0.042555,0.001888,0.004663
1,0,0.063829,0.000182,0.335127,0.025057,0.050146,0.003197,0.053958,0.000610,0.000107,...,0.029064,0.143391,0.080691,0.030758,0.036610,0.047451,0.007312,0.049544,0.010163,0.012916
2,0,0.003356,0.000189,0.311230,0.003716,0.010129,0.766956,0.010663,0.000244,0.000198,...,0.082572,0.334212,0.392698,0.104387,0.050050,1.000000,0.058840,0.147513,0.022522,0.273151
3,0,0.115109,0.000115,0.471762,0.017770,0.020854,0.000600,0.026637,0.000084,0.000194,...,0.030229,0.217920,0.046382,0.178885,0.033805,0.074702,0.007080,0.025919,0.006025,0.024615
4,0,0.056395,0.000074,0.108423,0.000458,0.001022,0.259142,0.015305,0.000029,0.000111,...,0.069734,0.184423,0.058655,0.073043,0.023322,0.073572,0.022898,0.075967,0.027190,0.026117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,1,0.083048,0.000139,0.396510,0.000845,0.002380,0.000666,0.016670,0.000482,0.000067,...,0.048064,0.183104,0.029973,0.048487,0.049980,0.038209,0.014787,0.026723,0.020401,0.028303
213,1,0.082018,0.000109,0.333452,0.000575,0.003664,0.007383,0.023933,0.000112,0.000004,...,0.048890,0.144090,0.012267,0.011086,0.054289,0.027532,0.029781,0.009441,0.001517,0.006231
214,1,0.048777,0.000299,0.508561,0.001760,0.003404,0.003056,0.012274,0.001220,0.000187,...,0.088055,0.292110,0.019608,0.098615,0.027909,0.089299,0.035945,0.028263,0.038490,0.018440
215,1,0.000793,0.000042,0.258038,0.026908,0.041186,0.045909,0.048794,0.000281,0.000251,...,0.072747,0.283006,0.110888,0.060092,0.040898,0.089001,0.017843,0.067326,0.060401,0.021957


% Positive class in Train = 35.86
% Positive class in Test  = 30.56


**Section 3.4 - Data Balancing with Missing Values Imputed**

The Synthetic Minority Oversampling Technique (SMOTE) is a statisical method that balances the amount of samples in a dataset by oversampling the minority class. Empty cells are imputed using the .nan_to_num() method.